In [1]:
DEVICE = "cpu"

MAX_WINDOWS = 3

In [2]:
from datasets import load_dataset
import pandas as pd
import numpy as np

dataset = load_dataset("Adilbai/stock-dataset")
df = dataset["train"].to_pandas()

df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values(["Ticker", "Date"])

# Choose stock
ticker = "AAPL"
stock_df = df[df["Ticker"] == ticker].copy()

stock_df.reset_index(drop=True, inplace=True)

/tmp/ipykernel_248100/3591124978.py:8: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df["Date"] = pd.to_datetime(df["Date"])


In [3]:
df["Ticker"].unique()[:20]
sp_df = df[df["Ticker"] == "^GSPC"][["Date", "Close"]]
sp_df = sp_df.rename(columns={"Close": "SP500_Close"})
stock_df = stock_df.merge(
    sp_df,
    on="Date",
    how="left"
)

In [4]:
import numpy as np
import pandas as pd

K_LIST = [120, 60, 40, 20, 10, 5, 1]  # keep within your history
HORIZON = 10
THRESHOLD = 0.02

# 1) daily returns for each stock
df = df.sort_values(["Ticker", "Date"]).copy()
df["ret_1d"] = df.groupby("Ticker")["Close"].pct_change()

# 2) market proxy return per date (cross-sectional mean)
mkt_ret = df.groupby("Date")["ret_1d"].mean().rename("mkt_ret_1d").reset_index()

# 3) choose one stock (AAPL here)
ticker = "AAPL"
stock_df = df[df["Ticker"] == ticker].sort_values("Date").copy()

# 4) merge market proxy return
stock_df = stock_df.merge(mkt_ret, on="Date", how="left")

# 5) build synthetic market "index price"
# Start at 1.0; cumprod of (1 + return)
stock_df["mkt_price"] = (1.0 + stock_df["mkt_ret_1d"].fillna(0.0)).cumprod()

def k_return(series, k):
    return (series / series.shift(k)) - 1.0

# 6) compute stock returns, market returns, and relative returns X_k
for k in K_LIST:
    stock_df[f"R_stock_{k}"] = k_return(stock_df["Close"], k)
    stock_df[f"R_mkt_{k}"]   = k_return(stock_df["mkt_price"], k)
    stock_df[f"X_{k}"]       = stock_df[f"R_stock_{k}"] - stock_df[f"R_mkt_{k}"]

In [5]:
# We use 1-day relative return X_1 as the “next-day relative return”
# and label y(i)=1 if max_{j=1..10} X_1(i+j) >= 2%
x1 = stock_df["X_1"].values
labels = np.zeros(len(stock_df), dtype=int)

for i in range(len(stock_df) - HORIZON):
    future_window = x1[i+1:i+1+HORIZON]
    labels[i] = int(np.nanmax(future_window) >= THRESHOLD)

stock_df["label"] = labels

# Trim edges explicitly (enough history for max K and enough future for label)
MIN_K = max(K_LIST)
stock_df = stock_df.iloc[MIN_K:-HORIZON].reset_index(drop=True)

In [6]:
feature_cols = [f"X_{k}" for k in K_LIST]
print("Rows:", len(stock_df))
print("NaNs in features:", stock_df[feature_cols].isna().sum().sum())
print(stock_df["label"].value_counts(normalize=True))
print(stock_df[feature_cols].head())

Rows: 1115
NaNs in features: 0
label
0    0.556054
1    0.443946
Name: proportion, dtype: float64
      X_120      X_60      X_40      X_20      X_10       X_5       X_1
0  0.116847  0.011193 -0.010283  0.066499  0.032245 -0.043683  0.002601
1  0.058908 -0.067213 -0.072030 -0.007428 -0.042156 -0.090005 -0.055046
2  0.099304 -0.100685  0.015939  0.014755 -0.050056 -0.053824  0.023682
3  0.075348 -0.074426  0.034212  0.039751 -0.031558 -0.032186  0.006616
4  0.074719 -0.102738 -0.022481  0.002488 -0.059369 -0.046698 -0.022435


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import numpy as np

X = stock_df[feature_cols].copy()
y = stock_df["label"].copy()

X_train_df, X_test_df, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

# Clean infinities (rare here, but safe)
X_train_df = X_train_df.replace([np.inf, -np.inf], np.nan)
X_test_df  = X_test_df.replace([np.inf, -np.inf], np.nan)

# Impute (median)
imputer = SimpleImputer(strategy="median")
X_train_i = imputer.fit_transform(X_train_df)
X_test_i  = imputer.transform(X_test_df)

# Scale
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train_i)
X_test_s  = scaler.transform(X_test_i)

print("Shapes:", X_train_s.shape, X_test_s.shape)
print("NaNs after impute:", np.isnan(X_train_s).sum(), np.isnan(X_test_s).sum())

Shapes: (892, 7) (223, 7)
NaNs after impute: 0 0


In [8]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(y_true, y_pred):
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "recall": recall_score(y_true, y_pred, zero_division=0),
        "f1": f1_score(y_true, y_pred, zero_division=0),
    }

def summarize_metrics(metrics_list):
    # metrics_list: list[dict]
    keys = metrics_list[0].keys()
    summary = {}
    for k in keys:
        vals = np.array([m[k] for m in metrics_list], dtype=float)
        summary[k] = {"mean": float(vals.mean()), "std": float(vals.std())}
    return summary


Important, area where we can define the number of windows.

In [9]:
import numpy as np

TRAIN_DAYS = 253
TEST_DAYS = 21
GAP = 10

def make_sliding_windows(df, train_days=253, gap=10, test_days=21):
    windows = []
    start = 0
    n = len(df)
    while start + train_days + gap + test_days <= n:
        train = df.iloc[start : start + train_days]
        test  = df.iloc[start + train_days + gap : start + train_days + gap + test_days]
        windows.append((train, test))
        start += test_days  # shift by 1 month (≈21 trading days)
    return windows

windows = make_sliding_windows(stock_df, TRAIN_DAYS, GAP, TEST_DAYS)
print("Number of windows:", len(windows))

MAX_WINDOWS = 3   # 🔁 change to 40 later if you want
windows = windows[:MAX_WINDOWS]

print("Windows used for evaluation:", len(windows))



Number of windows: 40
Windows used for evaluation: 3


SVM

In [10]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svm = SVC(kernel="rbf", C=1.0, gamma="scale", class_weight="balanced")
svm.fit(X_train_s, y_train)
preds = svm.predict(X_test_s)

print("SVM accuracy:", accuracy_score(y_test, preds))

SVM accuracy: 0.6098654708520179


In [11]:
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

svm_metrics = []

for train, test in windows:
    X_train_df = train[feature_cols].replace([np.inf, -np.inf], np.nan)
    y_train = train["label"].astype(int).values

    X_test_df = test[feature_cols].replace([np.inf, -np.inf], np.nan)
    y_test = test["label"].astype(int).values

    # Impute per-window
    imputer = SimpleImputer(strategy="median")
    X_train_i = imputer.fit_transform(X_train_df)
    X_test_i  = imputer.transform(X_test_df)

    # Scale per-window
    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train_i)
    X_test_s  = scaler.transform(X_test_i)

    svm = SVC(kernel="rbf", C=1.0, gamma="scale", class_weight="balanced")
    svm.fit(X_train_s, y_train)
    preds = svm.predict(X_test_s).astype(int)

    svm_metrics.append(compute_metrics(y_test, preds))

svm_summary = summarize_metrics(svm_metrics)
print("SVM metrics (mean ± std across windows):")
for k, v in svm_summary.items():
    print(f"{k}: {v['mean']:.4f} ± {v['std']:.4f}")

SVM metrics (mean ± std across windows):
accuracy: 0.4603 ± 0.0594
precision: 0.3436 ± 0.3362
recall: 0.2619 ± 0.2048
f1: 0.2456 ± 0.1789


RF

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

rf_metrics = []

for train, test in windows:
    X_train_df = train[feature_cols].replace([np.inf, -np.inf], np.nan)
    y_train = train["label"].astype(int).values

    X_test_df = test[feature_cols].replace([np.inf, -np.inf], np.nan)
    y_test = test["label"].astype(int).values

    # Impute per-window (no leakage)
    imputer = SimpleImputer(strategy="median")
    X_train = imputer.fit_transform(X_train_df)
    X_test  = imputer.transform(X_test_df)

    rf = RandomForestClassifier(
        n_estimators=1000,
        max_depth=5,
        max_features=min(len(feature_cols), 13),
        random_state=42,
        n_jobs=-1
    )
    rf.fit(X_train, y_train)
    preds = rf.predict(X_test).astype(int)

    rf_metrics.append(compute_metrics(y_test, preds))

rf_summary = summarize_metrics(rf_metrics)
print("RF metrics (mean ± std across windows):")
for k, v in rf_summary.items():
    print(f"{k}: {v['mean']:.4f} ± {v['std']:.4f}")

RF metrics (mean ± std across windows):
accuracy: 0.5238 ± 0.1166
precision: 0.7556 ± 0.3457
recall: 0.3915 ± 0.1963
f1: 0.3963 ± 0.0347


LSTM

In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def make_sequences(X, y, seq_len):
    Xs, ys = [], []
    for i in range(len(X) - seq_len):
        Xs.append(X[i:i+seq_len])
        ys.append(y[i+seq_len])
    return np.array(Xs), np.array(ys)

def eval_binary(y_true, y_pred):
    return {
        "acc": accuracy_score(y_true, y_pred),
        "prec": precision_score(y_true, y_pred, zero_division=0),
        "rec": recall_score(y_true, y_pred, zero_division=0),
        "f1": f1_score(y_true, y_pred, zero_division=0),
    }

In [14]:
def preprocess_window(train_df, test_df, feature_cols):
    # keep pandas here; avoid leakage by fitting only on train
    X_train_df = train_df[feature_cols].replace([np.inf, -np.inf], np.nan)
    X_test_df  = test_df[feature_cols].replace([np.inf, -np.inf], np.nan)

    y_train = train_df["label"].astype(int).values
    y_test  = test_df["label"].astype(int).values

    imputer = SimpleImputer(strategy="median")
    X_train_i = imputer.fit_transform(X_train_df)
    X_test_i  = imputer.transform(X_test_df)

    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train_i)
    X_test_s  = scaler.transform(X_test_i)

    return X_train_s, y_train, X_test_s, y_test

In [15]:
def preprocess_window(train_df, test_df, feature_cols):
    # keep pandas here; avoid leakage by fitting only on train
    X_train_df = train_df[feature_cols].replace([np.inf, -np.inf], np.nan)
    X_test_df  = test_df[feature_cols].replace([np.inf, -np.inf], np.nan)

    y_train = train_df["label"].astype(int).values
    y_test  = test_df["label"].astype(int).values

    imputer = SimpleImputer(strategy="median")
    X_train_i = imputer.fit_transform(X_train_df)
    X_test_i  = imputer.transform(X_test_df)

    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train_i)
    X_test_s  = scaler.transform(X_test_i)

    return X_train_s, y_train, X_test_s, y_test

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

def build_lstm_model(n_features, lstm_units=64, dropout=0.2):
    model = Sequential([
        LSTM(lstm_units, return_sequences=True, input_shape=(None, n_features)),
        Dropout(dropout),
        LSTM(lstm_units),
        Dense(1, activation="sigmoid")
    ])
    model.compile(optimizer="adam", loss="binary_crossentropy")
    return model

2026-02-08 19:01:47.252492: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-08 19:01:47.252718: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-08 19:01:47.277168: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-08 19:01:48.098310: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation or

In [17]:
SEQ_LEN = 20
EPOCHS = 30
BATCH_SIZE = 64

lstm_metrics = []
lstm_accs = []

for w_i, (train, test) in enumerate(windows, start=1):
    X_train_s, y_train, X_test_s, y_test = preprocess_window(train, test, feature_cols)

    X_train_seq, y_train_seq = make_sequences(X_train_s, y_train, SEQ_LEN)
    X_test_seq, y_test_seq   = make_sequences(X_test_s, y_test, SEQ_LEN)

    # If a window is too small (rare), skip it safely
    if len(X_train_seq) == 0 or len(X_test_seq) == 0:
        continue

    model = build_lstm_model(n_features=X_train_seq.shape[-1], lstm_units=64, dropout=0.2)

    es = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
    model.fit(
        X_train_seq, y_train_seq,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=0.1,
        verbose=0,
        callbacks=[es]
    )

    probs = model.predict(X_test_seq, verbose=0).ravel()
    preds = (probs >= 0.5).astype(int)

    m = eval_binary(y_test_seq, preds)
    lstm_metrics.append(m)
    lstm_accs.append(m["acc"])

print("LSTM mean acc:", float(np.mean(lstm_accs)))
print("LSTM std acc:", float(np.std(lstm_accs)))
print("LSTM mean f1:", float(np.mean([m['f1'] for m in lstm_metrics])))


2026-02-08 19:01:59.141329: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
/home/gurpreet/.conda/envs/lstm/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/home/gurpreet/.conda/envs/lstm/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/home/gurpreet/.conda/envs/lstm/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a laye

LSTM mean acc: 0.6666666666666666
LSTM std acc: 0.4714045207910317
LSTM mean f1: 0.3333333333333333


In [18]:
lstm_summary = {}
for k in ["acc", "prec", "rec", "f1"]:
    vals = np.array([m[k] for m in lstm_metrics], dtype=float)
    lstm_summary[k] = (float(vals.mean()), float(vals.std()))

print("lstm_summary (mean ± std across windows):")
for k, (mu, sd) in lstm_summary.items():
    print(f"{k}: {mu:.4f} ± {sd:.4f}")

lstm_summary (mean ± std across windows):
acc: 0.6667 ± 0.4714
prec: 0.3333 ± 0.4714
rec: 0.3333 ± 0.4714
f1: 0.3333 ± 0.4714


QLSTM

In [19]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

def compute_metrics(y_true, y_pred):
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "recall": recall_score(y_true, y_pred, zero_division=0),
        "f1": f1_score(y_true, y_pred, zero_division=0),
    }

def summarize(metrics_list):
    keys = metrics_list[0].keys()
    out = {}
    for k in keys:
        vals = np.array([m[k] for m in metrics_list], dtype=float)
        out[k] = (float(vals.mean()), float(vals.std()))
    return out

def preprocess_window(train_df, test_df, feature_cols):
    # pandas -> numpy, fit transforms ONLY on train (no leakage)
    X_train_df = train_df[feature_cols].replace([np.inf, -np.inf], np.nan)
    X_test_df  = test_df[feature_cols].replace([np.inf, -np.inf], np.nan)

    y_train = train_df["label"].astype(int).values
    y_test  = test_df["label"].astype(int).values

    imputer = SimpleImputer(strategy="median")
    X_train_i = imputer.fit_transform(X_train_df)
    X_test_i  = imputer.transform(X_test_df)

    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train_i)
    X_test_s  = scaler.transform(X_test_i)

    return X_train_s, y_train, X_test_s, y_test

In [20]:
def make_sequences(X, y, seq_len):
    Xs, ys = [], []
    for i in range(len(X) - seq_len):
        Xs.append(X[i:i+seq_len])
        ys.append(y[i+seq_len])
    return np.array(Xs), np.array(ys)

In [21]:
import torch
import torch.nn as nn
import pennylane as qml

def H_layer(nqubits):
    for idx in range(nqubits):
        qml.Hadamard(wires=idx)

def RY_layer(w):
    for idx, element in enumerate(w):
        qml.RY(element, wires=idx)

def entangling_layer(nqubits):
    for i in range(0, nqubits - 1, 2):
        qml.CNOT(wires=[i, i + 1])
    for i in range(1, nqubits - 1, 2):
        qml.CNOT(wires=[i, i + 1])

def q_function(x, q_weights, n_class):
    n_dep = q_weights.shape[0]
    n_qub = q_weights.shape[1]

    H_layer(n_qub)
    RY_layer(x)

    for k in range(n_dep):
        entangling_layer(n_qub)
        RY_layer(q_weights[k])

    return [qml.expval(qml.PauliZ(i)) for i in range(n_class)]

class TorchVQC(nn.Module):
    def __init__(self, vqc_depth, n_qubits, n_class):
        super().__init__()
        self.weights = nn.Parameter(0.01 * torch.randn(vqc_depth, n_qubits))
        self.dev = qml.device("default.qubit", wires=n_qubits)
        self.VQC = qml.QNode(q_function, self.dev, interface="torch")
        self.n_class = n_class

    def forward(self, X):
        # X: [B, n_qubits]
        # returns: [B, n_class]
        y_preds = torch.stack([torch.stack(self.VQC(x, self.weights, self.n_class)) for x in X])
        return y_preds.float()

In [22]:
class CustomQLSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, vqc_depth):
        super().__init__()
        self.hidden_size = hidden_size
        n_qubits = input_size + hidden_size

        self.input_gate  = TorchVQC(vqc_depth=vqc_depth, n_qubits=n_qubits, n_class=hidden_size)
        self.forget_gate = TorchVQC(vqc_depth=vqc_depth, n_qubits=n_qubits, n_class=hidden_size)
        self.cell_gate   = TorchVQC(vqc_depth=vqc_depth, n_qubits=n_qubits, n_class=hidden_size)
        self.output_gate = TorchVQC(vqc_depth=vqc_depth, n_qubits=n_qubits, n_class=hidden_size)

        self.output_post = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        h_prev, c_prev = hidden
        combined = torch.cat((x, h_prev), dim=1)  # [B, input+hidden]

        i_t = torch.sigmoid(self.input_gate(combined))
        f_t = torch.sigmoid(self.forget_gate(combined))
        g_t = torch.tanh(self.cell_gate(combined))
        o_t = torch.sigmoid(self.output_gate(combined))

        c_t = f_t * c_prev + i_t * g_t
        h_t = o_t * torch.tanh(c_t)

        out = self.output_post(h_t)  # [B, output_size]
        return out, h_t, c_t

class CustomLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, cell_module):
        super().__init__()
        self.hidden_size = hidden_size
        self.cell = cell_module

    def forward(self, x, hidden=None):
        # x: [B, T, input_size]
        B, T, _ = x.size()
        if hidden is None:
            h_t = torch.zeros(B, self.hidden_size, device=x.device)
            c_t = torch.zeros(B, self.hidden_size, device=x.device)
        else:
            h_t, c_t = hidden

        outputs = []
        for t in range(T):
            out, h_t, c_t = self.cell(x[:, t, :], (h_t, c_t))
            outputs.append(out.unsqueeze(1))  # [B,1,output_size]

        outputs = torch.cat(outputs, dim=1)  # [B,T,output_size]
        return outputs, (h_t, c_t)

In [23]:
import numpy as np
import torch
import torch.nn as nn

def train_qlstm_one_window(
    X_train_seq, y_train_seq, X_test_seq, y_test_seq,
    hidden_size=4, vqc_depth=1, lr=1e-2, epochs=2,
    batch_size=6, seed=0
):
    # Force CPU to avoid HIP "invalid device function"
    torch.manual_seed(seed)
    np.random.seed(seed)

    Xtr = torch.tensor(X_train_seq, dtype=torch.float32)
    ytr = torch.tensor(y_train_seq, dtype=torch.float32)
    Xte = torch.tensor(X_test_seq, dtype=torch.float32)

    input_size = Xtr.shape[-1]
    output_size = 1

    cell = CustomQLSTMCell(
        input_size=input_size,
        hidden_size=hidden_size,
        output_size=output_size,
        vqc_depth=vqc_depth
    )

    core = CustomLSTM(
        input_size=input_size,
        hidden_size=hidden_size,
        cell_module=cell
    )

    opt = torch.optim.Adam(core.parameters(), lr=lr)
    loss_fn = nn.BCEWithLogitsLoss()

    core.train()
    n = Xtr.shape[0]

    for ep in range(epochs):
        perm = torch.randperm(n)  # CPU permutation (no HIP kernel)
        for i in range(0, n, batch_size):
            idx = perm[i:i+batch_size]
            xb = Xtr[idx]
            yb = ytr[idx]

            opt.zero_grad()
            outputs, _ = core(xb)       # [B,T,1]
            logits = outputs[:, -1, 0]  # [B]
            loss = loss_fn(logits, yb)
            loss.backward()
            opt.step()

    core.eval()
    with torch.no_grad():
        out_te, _ = core(Xte)
        logits_te = out_te[:, -1, 0]
        probs = torch.sigmoid(logits_te).numpy()
        preds = (probs >= 0.5).astype(int)

    return preds, probs

This QLSTM cannot accurately be compared with htun's lstm, but we can compare their implementations.

Htun et al. used  ~494 stocks and 23 windows

We are using 1 stock and 3 windows for testing throughout in the first runthrough.

In [24]:
qlstm_metrics = []
SEQ_LEN = 10

for w_i, (train, test) in enumerate(windows[:MAX_WINDOWS], start=1):
    X_train_s, y_train, X_test_s, y_test = preprocess_window(train, test, feature_cols)

    X_train_seq, y_train_seq = make_sequences(X_train_s, y_train, SEQ_LEN)
    X_test_seq, y_test_seq   = make_sequences(X_test_s, y_test, SEQ_LEN)

    if len(X_train_seq) == 0 or len(X_test_seq) == 0:
        continue

    preds, probs = train_qlstm_one_window(
        X_train_seq, y_train_seq, X_test_seq, y_test_seq,
        hidden_size=4, vqc_depth=1, lr=1e-2, epochs=4, batch_size=8, seed=42
    )

    m = compute_metrics(y_test_seq, preds)
    qlstm_metrics.append(m)


In [25]:
len(qlstm_metrics), qlstm_metrics[:2]

(3,
 [{'accuracy': 0.2727272727272727, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0},
  {'accuracy': 0.8181818181818182,
   'precision': 0.8181818181818182,
   'recall': 1.0,
   'f1': 0.9}])

In [26]:
def newsummarize(metrics_list):
    keys = metrics_list[0].keys()
    out = {}
    for k in keys:
        vals = np.array([m[k] for m in metrics_list], dtype=float)
        out[k] = (float(vals.mean()), float(vals.std()))
    return out

In [27]:
qlstm_summary = newsummarize(qlstm_metrics)
print("QLSTM metrics (mean ± std across evaluated windows):")
for k, (mu, sd) in qlstm_summary.items():
    print(f"{k}: {mu:.4f} ± {sd:.4f}")

QLSTM metrics (mean ± std across evaluated windows):
accuracy: 0.6364 ± 0.2571
precision: 0.6061 ± 0.4349
recall: 0.5333 ± 0.4110
f1: 0.5500 ± 0.3937


TORCH VQC

In [28]:
class TorchVQC(nn.Module):
	def __init__(self, vqc_depth, n_qubits, n_class):
		super().__init__()
		self.weights = nn.Parameter(0.01 * torch.randn(vqc_depth, n_qubits))  # g rotation params
		self.dev = qml.device("default.qubit", wires=n_qubits)  # Can use different simulation backend or quantum computers.
		self.VQC = qml.QNode(q_function, self.dev, interface = "torch")

		self.n_class = n_class


	def forward(self, X):
		y_preds = torch.stack([torch.stack(self.VQC(x, self.weights, self.n_class)).float() for x in X]) # PennyLane 0.35.1
		return y_preds

In [29]:
class VQCFeedForwardClassifier(nn.Module):
    """
    Feedforward classifier: X (batch, F) -> binary logit.
    Projects features to latent_dim, runs TorchVQC, then outputs 1 logit.
    """
    def __init__(self, in_dim, latent_dim=8, vqc_depth=2):
        super().__init__()
        self.latent_dim = latent_dim
        self.pre = nn.Linear(in_dim, latent_dim)
        self.vqc = TorchVQC(vqc_depth=vqc_depth, n_qubits=latent_dim, n_class=latent_dim)
        self.post = nn.Linear(latent_dim, 1)

    def forward(self, x):
        # x: (B, F)
        z = torch.tanh(self.pre(x))
        z = torch.tanh(self.vqc(z))   # (B, latent_dim)
        logit = self.post(z).squeeze(-1)  # (B,)
        return logit


def train_vqc_ff_one_window(X_train_s, y_train, X_test_s, y_test,
                            epochs=2, batch_size=32, lr=1e-2,
                            latent_dim=8, vqc_depth=2, seed=0):
    torch.manual_seed(seed)
    np.random.seed(seed)

    device = "cpu"  # keep CPU stable for PennyLane
    Xtr = torch.tensor(X_train_s, dtype=torch.float32, device=device)
    ytr = torch.tensor(y_train.astype(np.float32), dtype=torch.float32, device=device)
    Xte = torch.tensor(X_test_s, dtype=torch.float32, device=device)

    model = VQCFeedForwardClassifier(in_dim=Xtr.shape[1], latent_dim=latent_dim, vqc_depth=vqc_depth).to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.BCEWithLogitsLoss()

    n = Xtr.shape[0]

    for ep in range(1, epochs+1):
        model.train()
        perm = torch.randperm(n)  # CPU-safe
        losses = []

        for i in range(0, n, batch_size):
            idx = perm[i:i+batch_size]
            xb = Xtr[idx]
            yb = ytr[idx]

            opt.zero_grad()
            logits = model(xb)
            loss = loss_fn(logits, yb)
            loss.backward()
            opt.step()
            losses.append(loss.item())

        print(f"[TorchVQC-FF] epoch {ep}/{epochs} train_loss={np.mean(losses):.6f}")

    model.eval()
    with torch.no_grad():
        logits = model(Xte).cpu().numpy()
        probs = 1 / (1 + np.exp(-logits))
        preds = (probs >= 0.5).astype(int)

    return preds, probs


In [30]:
VQC_EPOCHS = 2
VQC_BATCH  = 32

vqc_ff_metrics = []
vqc_ff_accs = []

for w_i, (train, test) in enumerate(windows, start=1):
    X_train_s, y_train, X_test_s, y_test = preprocess_window(train, test, feature_cols)

    preds, probs = train_vqc_ff_one_window(
        X_train_s, y_train, X_test_s, y_test,
        epochs=VQC_EPOCHS, batch_size=VQC_BATCH, lr=1e-2,
        latent_dim=8, vqc_depth=2, seed=42
    )

    m = eval_binary(y_test, preds)
    vqc_ff_metrics.append(m)
    vqc_ff_accs.append(m["acc"])

vqc_ff_summary = {k: (float(np.mean([m[k] for m in vqc_ff_metrics])),
                      float(np.std([m[k] for m in vqc_ff_metrics])))
                  for k in ["acc","prec","rec","f1"]}

print("TorchVQC-FF summary (mean ± std across windows):")
for k, (mu, sd) in vqc_ff_summary.items():
    print(f"{k}: {mu:.4f} ± {sd:.4f}")


[TorchVQC-FF] epoch 1/2 train_loss=0.690107
[TorchVQC-FF] epoch 2/2 train_loss=0.671609
[TorchVQC-FF] epoch 1/2 train_loss=0.686205
[TorchVQC-FF] epoch 2/2 train_loss=0.667958
[TorchVQC-FF] epoch 1/2 train_loss=0.679398
[TorchVQC-FF] epoch 2/2 train_loss=0.664912
TorchVQC-FF summary (mean ± std across windows):
acc: 0.3810 ± 0.1029
prec: 0.3929 ± 0.2592
rec: 0.5132 ± 0.3678
f1: 0.3716 ± 0.1792


FWP VQC

In [31]:
### VQC

def H_layer(nqubits):
	"""Layer of single-qubit Hadamard gates.
	"""
	for idx in range(nqubits):
		qml.Hadamard(wires=idx)

def RX_layer(w):
	"""Layer of parametrized qubit rotations around the y axis.
	"""
	for idx, element in enumerate(w):
		qml.RX(element, wires=idx)

def RY_layer(w):
	"""Layer of parametrized qubit rotations around the y axis.
	"""
	for idx, element in enumerate(w):
		qml.RY(element, wires=idx)

def RZ_layer(w):
	"""Layer of parametrized qubit rotations around the y axis.
	"""
	for idx, element in enumerate(w):
		qml.RZ(element, wires=idx)




def entangling_layer(nqubits):
	"""Layer of CNOTs followed by another shifted layer of CNOT.
	"""
	# In other words it should apply something like :
	# CNOT  CNOT  CNOT  CNOT...  CNOT
	#   CNOT  CNOT  CNOT...  CNOT
	for i in range(0, nqubits - 1, 2):  # Loop over even indices: i=0,2,...N-2
		qml.CNOT(wires=[i, i + 1])
	for i in range(1, nqubits - 1, 2):  # Loop over odd indices:  i=1,3,...N-3
		qml.CNOT(wires=[i, i + 1])
	

def cycle_entangling_layer(nqubits):
	for i in range(0, nqubits):  
		qml.CNOT(wires=[i, (i + 1) % nqubits])



##############



### VQC function

def quantum_net(inputs, q_weights, n_outputs):
	n_dep = q_weights.shape[0]
	n_qub = q_weights.shape[1]

	H_layer(n_qub)

	RY_layer(inputs)

	for k in range(n_dep):
		entangling_layer(n_qub)
		RY_layer(q_weights[k])


	return [qml.expval(qml.PauliZ(position)) for position in range(n_outputs)]


##############

class HideSignature:
	def __init__(self, partial_func):
		self.partial_func = partial_func

	def __call__(self, inputs, q_weights):
		return self.partial_func(inputs, q_weights)

##############

### VQC batch wrapper

class BatchVQC:
	def __init__(self, q_func):
		self.q_func = q_func

	def __call__(self, inputs, q_weights):

		res_all = []
		for input_item, q_weight_item in zip(inputs, q_weights):
			res = self.q_func(input_item, q_weight_item) 
			res_all.append(torch.stack(res)) 

		return torch.stack(res_all)


##############

In [32]:
from functools import partial

# --- reuse your H_layer, RY_layer, entangling_layer, quantum_net, HideSignature, BatchVQC exactly ---
# (Make sure they are defined)

class FWPCellCls(nn.Module):
    def __init__(self, s_dim, a_dim=4, latent_dim=8, q_depth=2):
        super().__init__()
        self.n_qubits = latent_dim
        self.q_depth = q_depth

        dev = qml.device("default.qubit", wires=self.n_qubits)
        self.q_func = BatchVQC(qml.QNode(HideSignature(partial(quantum_net, n_outputs=a_dim)), dev, interface="torch"))

        self.slow_program_encoder = nn.Linear(s_dim, latent_dim)
        self.slow_program_layer_idx = nn.Linear(latent_dim, self.q_depth)
        self.slow_program_qubit_idx = nn.Linear(latent_dim, self.n_qubits)

        # Output a single LOGIT
        self.post_processing = nn.Linear(a_dim, 1)

    def forward(self, batch_item, previous_circuit_param):
        # batch_item: (B, s_dim)
        res = self.slow_program_encoder(batch_item)
        res_layer_idx = self.slow_program_layer_idx(res)
        res_qubit_idx = self.slow_program_qubit_idx(res)

        out_circuit_params = []
        for layer_idx, qubit_idx in zip(res_layer_idx, res_qubit_idx):
            out_circuit_params.append(torch.outer(layer_idx, qubit_idx))
        out_circuit_params = torch.stack(out_circuit_params)

        out_circuit_params = out_circuit_params + previous_circuit_param

        # quantum output: (B, a_dim)
        q_out = self.q_func(batch_item, out_circuit_params)

        logit = self.post_processing(q_out).squeeze(-1)  # (B,)
        return logit, out_circuit_params

    def initial_fast_params(self, batch_size):
        return torch.zeros(batch_size, self.q_depth, self.n_qubits)


class FWPCls(nn.Module):
    def __init__(self, s_dim, a_dim=4, latent_dim=8, q_depth=2):
        super().__init__()
        self.cell = FWPCellCls(s_dim=s_dim, a_dim=a_dim, latent_dim=latent_dim, q_depth=q_depth)

    def forward(self, x_seq):
        # x_seq: (B, T, s_dim)
        B, T, _ = x_seq.shape
        fast = self.cell.initial_fast_params(B)

        logits_over_time = []
        for t in range(T):
            logit_t, fast = self.cell(x_seq[:, t, :], fast)
            logits_over_time.append(logit_t)

        # (T, B)
        return torch.stack(logits_over_time, dim=0)


In [ ]:
from functools import partial

# ---------- VQC building blocks ----------
def H_layer(nqubits):
    for idx in range(nqubits):
        qml.Hadamard(wires=idx)

def RY_layer(w):
    for idx, element in enumerate(w):
        qml.RY(element, wires=idx)

def entangling_layer(nqubits):
    for i in range(0, nqubits - 1, 2):
        qml.CNOT(wires=[i, i + 1])
    for i in range(1, nqubits - 1, 2):
        qml.CNOT(wires=[i, i + 1])

def quantum_net(inputs, q_weights, n_outputs):
    n_dep = q_weights.shape[0]
    n_qub = q_weights.shape[1]

    H_layer(n_qub)
    RY_layer(inputs)

    for k in range(n_dep):
        entangling_layer(n_qub)
        RY_layer(q_weights[k])

    return [qml.expval(qml.PauliZ(i)) for i in range(n_outputs)]

class HideSignature:
    def __init__(self, partial_func):
        self.partial_func = partial_func
    def __call__(self, inputs, q_weights):
        return self.partial_func(inputs, q_weights)

class BatchVQC:
    """
    inputs:   (B, n_qubits)
    q_weights:(B, q_depth, n_qubits)
    returns:  (B, a_dim)
    """
    def __init__(self, qnode):
        self.qnode = qnode

    def __call__(self, inputs, q_weights):
        outs = []
        for x_i, w_i in zip(inputs, q_weights):
            res = self.qnode(x_i, w_i)           # list length a_dim
            outs.append(torch.stack(res))        # (a_dim,)
        return torch.stack(outs)                 # (B, a_dim)

# ---------- FWP ----------
class FWPCell(nn.Module):
    def __init__(self, s_dim, a_dim=4, latent_dim=8, q_depth=2):
        super().__init__()
        self.n_qubits = latent_dim
        self.q_depth = q_depth
        self.a_dim = a_dim

        dev = qml.device("default.qubit", wires=self.n_qubits)

        qnode = qml.QNode(
            HideSignature(partial(quantum_net, n_outputs=a_dim)),
            dev,
            interface="torch"
        )
        self.q_func = BatchVQC(qnode)

        # Encode s_dim -> latent_dim (n_qubits)
        self.slow_program_encoder = nn.Linear(s_dim, latent_dim)
        self.slow_program_layer_idx = nn.Linear(latent_dim, q_depth)
        self.slow_program_qubit_idx = nn.Linear(latent_dim, latent_dim)

        # Post-process a_dim -> 1 logit
        self.post_processing = nn.Linear(a_dim, 1)

    def initial_fast_params(self, batch_size, device):
        return torch.zeros(batch_size, self.q_depth, self.n_qubits, device=device, dtype=torch.float32)

    def forward(self, batch_item, prev_params):
        """
        batch_item: (B, s_dim)
        prev_params:(B, q_depth, n_qubits)
        """
        z = self.slow_program_encoder(batch_item)            # (B, latent_dim)
        layer_idx = self.slow_program_layer_idx(z)           # (B, q_depth)
        qubit_idx = self.slow_program_qubit_idx(z)           # (B, n_qubits)

        # Outer product per batch: (q_depth, n_qubits)
        out_params = torch.stack([torch.outer(l, q) for l, q in zip(layer_idx, qubit_idx)], dim=0)
        out_params = out_params + prev_params                # (B, q_depth, n_qubits)
        out_params = out_params.float()

        # Quantum eval expects inputs shaped (B, n_qubits)
        # So ensure we feed encoded features (z) into circuit inputs
        
        # If prefer raw batch_item, need s_dim == n_qubits.
        vqc_in = z                                            # (B, n_qubits)
        vqc_out = self.q_func(vqc_in, out_params).float()  # <-- FORCE float32
        logit = self.post_processing(vqc_out)              # now float32 x float32

        #logit = self.post_processing(vqc_out)                  # (B, 1)
        return logit, out_params

class FWP(nn.Module):
    """
    Sequence model. Input: (B, T, s_dim). Output: (T, B, 1) logits.
    """
    def __init__(self, s_dim, a_dim=4, latent_dim=8, q_depth=2):
        super().__init__()
        self.cell = FWPCell(s_dim=s_dim, a_dim=a_dim, latent_dim=latent_dim, q_depth=q_depth)

    def forward(self, x):
        # x: (B, T, s_dim)
        B, T, _ = x.shape
        device = x.device
        params = self.cell.initial_fast_params(B, device=device)
        

        outs = []
        for t in range(T):
            logit, params = self.cell(x[:, t, :], params)  # logit: (B,1)
            outs.append(logit)
        return torch.stack(outs, dim=0)  # (T, B, 1)


In [44]:
def train_fwp_one_window(X_train_seq, y_train_seq, X_test_seq, y_test_seq,
                         epochs=2, batch_size=16, lr=1e-2,
                         a_dim=4, latent_dim=8, q_depth=2, seed=42,
                         device="cpu"):

    import numpy as np
    import torch
    import torch.nn as nn

    torch.manual_seed(seed)
    np.random.seed(seed)

    # ---- FORCE FLOAT32 ----
    Xtr = torch.tensor(X_train_seq, dtype=torch.float32, device=device)
    ytr = torch.tensor(y_train_seq, dtype=torch.float32, device=device)
    Xte = torch.tensor(X_test_seq,  dtype=torch.float32, device=device)
    yte = torch.tensor(y_test_seq,  dtype=torch.float32, device=device)

    # Build model (must match dtype)
    model = FWP(s_dim=Xtr.shape[-1], a_dim=a_dim, latent_dim=latent_dim, q_depth=q_depth).to(device).float()

    opt = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.BCEWithLogitsLoss()

    n = Xtr.shape[0]
    model.train()
    for ep in range(epochs):
        perm = torch.randperm(n, device=device)
        ep_loss = 0.0

        for i in range(0, n, batch_size):
            idx = perm[i:i+batch_size]
            xb = Xtr[idx]   # (B, T, F) float32
            yb = ytr[idx]   # (B,) float32

            opt.zero_grad()
            logits_TB = model(xb)          # should return (T, B, 1) or (T, B)
            logits_last = logits_TB[-1].squeeze(-1)  # (B,)
            loss = loss_fn(logits_last, yb)
            loss.backward()
            opt.step()

            ep_loss += loss.item() * len(idx)

        print(f"[FWP] epoch {ep+1}/{epochs} train_loss={ep_loss/n:.6f}")

    # Eval
    model.eval()
    with torch.no_grad():
        logits_TB = model(Xte)
        logits_last = logits_TB[-1].squeeze(-1)
        probs = torch.sigmoid(logits_last).cpu().numpy()
        preds = (probs >= 0.5).astype(int)

    return preds, probs


In [45]:
FWP_EPOCHS = 2
FWP_BATCH  = 16
SEQ_LEN    = 20  # keep same as your LSTM seq_len if you want parity

fwp_metrics = []
fwp_accs = []

for w_i, (train, test) in enumerate(windows, start=1):
    
    X_train_s, y_train, X_test_s, y_test = preprocess_window(train, test, feature_cols)

    X_train_seq, y_train_seq = make_sequences(X_train_s, y_train, SEQ_LEN)
    X_test_seq,  y_test_seq  = make_sequences(X_test_s,  y_test,  SEQ_LEN)

    if len(X_train_seq) == 0 or len(X_test_seq) == 0:
        continue

    preds, probs = train_fwp_one_window(
        X_train_seq, y_train_seq, X_test_seq, y_test_seq,
        epochs=FWP_EPOCHS, batch_size=FWP_BATCH, lr=1e-2,
        a_dim=4, latent_dim=8, q_depth=2, seed=42
    )

    m = eval_binary(y_test_seq, preds)
    fwp_metrics.append(m)
    fwp_accs.append(m["acc"])

fwp_summary = {k: (float(np.mean([m[k] for m in fwp_metrics])),
                   float(np.std([m[k] for m in fwp_metrics])))
               for k in ["acc","prec","rec","f1"]}

print("FWP summary (mean ± std across windows):")
for k, (mu, sd) in fwp_summary.items():
    print(f"{k}: {mu:.4f} ± {sd:.4f}")


[FWP] epoch 1/2 train_loss=0.682812
[FWP] epoch 2/2 train_loss=0.625217
[FWP] epoch 1/2 train_loss=0.683713
[FWP] epoch 2/2 train_loss=0.686386
[FWP] epoch 1/2 train_loss=0.608857
[FWP] epoch 2/2 train_loss=0.579451
FWP summary (mean ± std across windows):
acc: 0.6667 ± 0.4714
prec: 0.3333 ± 0.4714
rec: 0.3333 ± 0.4714
f1: 0.3333 ± 0.4714


Summaries

In [46]:
def print_summary_table(name_to_summary):
    # name_to_summary: dict[str, dict(metric -> (mean,std))]
    metrics = ["accuracy", "precision", "recall", "f1"]
    print("Model".ljust(10), *(m.ljust(20) for m in metrics))
    for name, summ in name_to_summary.items():
        row = [name.ljust(10)]
        for m in metrics:
            mu, sd = summ[m]
            row.append(f"{mu:.4f} ± {sd:.4f}".ljust(20))
        print(*row)

# name_to_summary = {"RF": rf_summary, "SVM": svm_summary, "LSTM": lstm_summary, "QLSTM": qlstm_summary, ...}
# print_summary_table(name_to_summary)

In [47]:
name_to_summary = {"RF": rf_summary, "SVM": svm_summary, "LSTM": lstm_summary, "QLSTM": qlstm_summary}
#print_summary_table(name_to_summary)